In [21]:
import matplotlib.pyplot as plt
import pandas
import numpy as np
from config import input_datadir, result_datadir
import os
import fnmatch
import ARX_Optimization

In [22]:
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

In [38]:
# Convert from pt to inches
inches_per_pt = 1 / 72.27

# Golden ratio to set aesthetic figure height
golden_ratio = (5**.5 - 1) / 2

# Figure width in inches
fig_width_in = 390 * inches_per_pt
# Figure height in inches
fig_height_in = fig_width_in * golden_ratio

In [23]:
from IPython.core.display import display, HTML
display(HTML("<style>"
    + "#notebook { padding-top:0px !important; } " 
    + ".container { width:100% !important; } "
    + ".end_space { min-height:0px !important; } "
    + "</style>"))

In [24]:
# Specify Order

m = 2
n = 2

In [32]:
def plot_formatting(data, idx, outlier_run, maxtime_name):
    dfd_maxtime, sim_clean, maxrun, sim_maxtime, dfd_outlier, output_outlier, sim_outlier, sigmas, runtime, error, max_error = data
    ax[0,0].set_title('Process Data and identification technique simulation')
    ax[0,0].set_ylabel('Process Output')
    
    ax[1,1].set_title('Absolute difference in coefficients of clean versus noisy data.')
    ax[1,1].set_ylabel('Error')
    
    ax[1,0].set_title('Data, and ARX_approximation from identification.')
    ax[1,0].set_xlabel('Time (s)')
    ax[1,0].set_ylabel('Process Output')
    
    ax[0,1].set_title('Time elapsed during the determination of coefficients.')
    ax[0,1].set_xlabel('standard deviation')
    ax[0,1].set_ylabel('Runtime')
    
    ax[0,0].plot(dfd_maxtime[0].values, dfd_maxtime[1].values, color='b', label='Clean Data')
    ax[0,0].plot(dfd_maxtime[0].values, sim_clean, color='k', label='Clean approximation')
    ax[0,0].plot(dfd_maxtime[0].values, maxrun.values, color='grey', zorder=0, label = 'Slowest runtime Data')  # "('+'_'+os.path.splitext(maxtime_name)[0])"
    ax[0,0].plot(dfd_maxtime[0].values, sim_maxtime, '--', color='r', label='Slowest runtime approximation')

    ax[1,0].plot(dfd_outlier[0].values, output_outlier, color='k', alpha=0.5, label = 'Error Outlier Output') #+'_'+os.path.splitext(outlier_run)[0])
    ax[1,0].plot(dfd_outlier[0].values, sim_outlier, '--', color='k',label='Largest error approximation')

    ax[1,1].scatter(sigmas, error, color='k')
    ax[1,1].scatter(sigmas[idx], max_error, color='red', label= 'Largest Error') #os.path.splitext(outlier_run)[0])

    ax[0,1].scatter(sigmas, runtime, color='k')
    ax[0,1].scatter(sigmas[runtime.index(max(runtime))], max(runtime), color='red')

    ax[0,0].legend()
    ax[1,0].legend()
    ax[1,1].legend()
    path = 'C:/Users/annam/Desktop/Johanna/CSC411/Report/IMAGES'
#     fig.savefig(os.path.join(path,os.path.splitext(outlier_run)[0])+'.pdf')


In [33]:
def do_sim(ydata, data, pt, ptr):
    dfr_maxtime, dfr_outlier, dfd_maxtime, dfd_outlier = ydata
    sim_clean = ARX_Optimization.simulate(dfr_maxtime.loc[0][:m+n].values, data.Q1.values, m, n) + dfd_maxtime[1][0] #change Q1 to work for any data set or code specific process inputs
    sim_maxtime = ARX_Optimization.simulate(dfr_maxtime.loc[ptr][:m+n].values, data.Q1.values, m, n) + dfd_maxtime[1][0] #change Q1 to work for any data set or code specific process inputs
    sim_outlier = ARX_Optimization.simulate(dfr_outlier.loc[pt][:m+n].values, data.Q1.values, m, n) + dfd_outlier[1][0] #change Q1 to work for any data set or code specific process inputs
    return sim_clean, sim_maxtime, sim_outlier
    

In [34]:
def max_er_rt(error, runtime, sigmas, max_error, max_runtime, dfr, m, n):
    prev_max_error = max_error
    prev_max_runtime = max_runtime
    for i in range(0,len(dfr)):
        difference=[] 
        for j in range(m+n):
            difference.append(abs(dfr[j][0] - dfr[j][i]))
        error.append(sum(difference))
        runtime.append(dfr[j+1][i])
        sigmas.append(dfr[j+2][i])
        max_error = max(error)
        max_runtime = max(runtime)
    return error, runtime, sigmas, max_error, max_runtime, prev_max_error, prev_max_runtime

In [35]:
# def mx_Er_test(max_error, prev_max_error, resultname):
#     if max_error > prev_max_error:
#         idx = error.index(max_error)
#         outlier_run = resultname
#         dfd_outlier = extract_data(outlier_run)
#         if idx <= 9:
#             pt = idx
#         else:
#             pt = int(str(idx)[1])
#         output_outlier = dfd_outlier[pt]
#     else:
#         pass
#     return pt, idx, outlier_run, output_outlier

In [36]:
def extract_data(filename):
    dfd = dfd_outlier = pandas.read_excel(result_datadir/filename, sheet_name='Input_Data', header=None)
    dfr = pandas.read_excel(result_datadir/filename, sheet_name='ID_Results', header=None)
    return dfd, dfr

In [39]:
# for method, method_name in zip(ARX_Optimization.ID_techs, ARX_Optimization.ID_tech_names): 
method_name=ARX_Optimization.ID_tech_names[0]
for dataname in os.listdir(input_datadir):  # Get data filename
    if dataname.endswith('.csv'):
        fig, ax = plt.subplots(2,2, figsize=(fig_width_in,fig_height_in))
        error = []
        runtime = []
        sigmas = []
        max_error = max_runtime = 0
        for resultname in os.listdir(result_datadir): # Get result filename
            if resultname.endswith('.xlsx') and fnmatch.fnmatch(os.path.splitext(resultname)[0], '*'+os.path.splitext(dataname)[0]+'*') and fnmatch.fnmatch(os.path.splitext(resultname)[0], '*'+method_name+'*') and fnmatch.fnmatch(os.path.splitext(resultname)[0], '*m'+str(m)+'_n'+str(n)+'*'): #group result filenames that match the data filename and ID technique
                dfd, dfr = extract_data(resultname)
                data = pandas.read_csv(input_datadir/dataname)

                error, runtime, sigmas, max_error, max_runtime, prev_max_error, prev_max_runtime = max_er_rt(error, runtime, sigmas, max_error, max_runtime, dfr, m, n)

                if max_error > prev_max_error:
                    idx = error.index(max_error)
                    outlier_run = resultname
                    dfd_outlier, dfr_outlier = extract_data(outlier_run)
                    if idx <= 9:
                        pt = idx
                    else:
                        pt = int(str(idx)[1])
                    output_outlier = dfd_outlier[pt]

                if max_runtime > prev_max_runtime:
                    idxr = runtime.index(max_runtime)
                    maxtime_name = resultname
                    dfd_maxtime, dfr_maxtime = extract_data(maxtime_name)

                    if idxr <= 9:
                        ptr = idxr
                    else:
                        ptr = int(str(idxr)[1])
                        if ptr == 0:
                            ptr = 1
                    maxrun = dfd_maxtime[ptr]

        sim_clean, sim_maxtime, sim_outlier = do_sim([dfr_maxtime, dfr_outlier, dfd_maxtime, dfd_outlier], data, pt, ptr)
        data_lst = [dfd_maxtime, sim_clean, maxrun, sim_maxtime, dfd_outlier, output_outlier, sim_outlier, sigmas, runtime, error, max_error]
        plot_formatting(data_lst, idx, outlier_run, maxtime_name)




Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x000001C9F7D01378> (for post_execute):


RuntimeError: latex was not able to process the following string:
b'Data, and ARX_approximation from identification.'

Here is the full report generated by latex:
This is pdfTeX, Version 3.14159265-2.6-1.40.20 (MiKTeX 2.9.7000 64-bit)
entering extended mode
(C:/Users/annam/.matplotlib/tex.cache/603d518dbcd1d75a92e7b09664c078a8.tex
LaTeX2e <2018-12-01>
("C:\Users\annam\AppData\Local\Programs\MiKTeX 2.9\tex/latex/base\article.cls"
Document Class: article 2018/09/03 v1.4i Standard LaTeX document class
("C:\Users\annam\AppData\Local\Programs\MiKTeX 2.9\tex/latex/base\size10.clo"))

("C:\Users\annam\AppData\Local\Programs\MiKTeX 2.9\tex/latex/type1cm\type1cm.st
y")
("C:\Users\annam\AppData\Local\Programs\MiKTeX 2.9\tex/latex/base\textcomp.sty"
("C:\Users\annam\AppData\Local\Programs\MiKTeX 2.9\tex/latex/base\ts1enc.def"
("C:\Users\annam\AppData\Local\Programs\MiKTeX 2.9\tex/latex/base\ts1enc.dfu"))
) ("C:\Users\annam\AppData\Local\Programs\MiKTeX 2.9\tex/latex/base\inputenc.st

RuntimeError: latex was not able to process the following string:
b'Data, and ARX_approximation from identification.'

Here is the full report generated by latex:
This is pdfTeX, Version 3.14159265-2.6-1.40.20 (MiKTeX 2.9.7000 64-bit)
entering extended mode
(C:/Users/annam/.matplotlib/tex.cache/603d518dbcd1d75a92e7b09664c078a8.tex
LaTeX2e <2018-12-01>
("C:\Users\annam\AppData\Local\Programs\MiKTeX 2.9\tex/latex/base\article.cls"
Document Class: article 2018/09/03 v1.4i Standard LaTeX document class
("C:\Users\annam\AppData\Local\Programs\MiKTeX 2.9\tex/latex/base\size10.clo"))

("C:\Users\annam\AppData\Local\Programs\MiKTeX 2.9\tex/latex/type1cm\type1cm.st
y")
("C:\Users\annam\AppData\Local\Programs\MiKTeX 2.9\tex/latex/base\textcomp.sty"
("C:\Users\annam\AppData\Local\Programs\MiKTeX 2.9\tex/latex/base\ts1enc.def"
("C:\Users\annam\AppData\Local\Programs\MiKTeX 2.9\tex/latex/base\ts1enc.dfu"))
) ("C:\Users\annam\AppData\Local\Programs\MiKTeX 2.9\tex/latex/base\inputenc.st

<Figure size 388.543x240.133 with 4 Axes>

RuntimeError: latex was not able to process the following string:
b'Data, and ARX_approximation from identification.'

Here is the full report generated by latex:
This is pdfTeX, Version 3.14159265-2.6-1.40.20 (MiKTeX 2.9.7000 64-bit)
entering extended mode
(C:/Users/annam/.matplotlib/tex.cache/603d518dbcd1d75a92e7b09664c078a8.tex
LaTeX2e <2018-12-01>
("C:\Users\annam\AppData\Local\Programs\MiKTeX 2.9\tex/latex/base\article.cls"
Document Class: article 2018/09/03 v1.4i Standard LaTeX document class
("C:\Users\annam\AppData\Local\Programs\MiKTeX 2.9\tex/latex/base\size10.clo"))

("C:\Users\annam\AppData\Local\Programs\MiKTeX 2.9\tex/latex/type1cm\type1cm.st
y")
("C:\Users\annam\AppData\Local\Programs\MiKTeX 2.9\tex/latex/base\textcomp.sty"
("C:\Users\annam\AppData\Local\Programs\MiKTeX 2.9\tex/latex/base\ts1enc.def"
("C:\Users\annam\AppData\Local\Programs\MiKTeX 2.9\tex/latex/base\ts1enc.dfu"))
) ("C:\Users\annam\AppData\Local\Programs\MiKTeX 2.9\tex/latex/base\inputenc.st

<Figure size 388.543x240.133 with 4 Axes>

RuntimeError: latex was not able to process the following string:
b'Data, and ARX_approximation from identification.'

Here is the full report generated by latex:
This is pdfTeX, Version 3.14159265-2.6-1.40.20 (MiKTeX 2.9.7000 64-bit)
entering extended mode
(C:/Users/annam/.matplotlib/tex.cache/603d518dbcd1d75a92e7b09664c078a8.tex
LaTeX2e <2018-12-01>
("C:\Users\annam\AppData\Local\Programs\MiKTeX 2.9\tex/latex/base\article.cls"
Document Class: article 2018/09/03 v1.4i Standard LaTeX document class
("C:\Users\annam\AppData\Local\Programs\MiKTeX 2.9\tex/latex/base\size10.clo"))

("C:\Users\annam\AppData\Local\Programs\MiKTeX 2.9\tex/latex/type1cm\type1cm.st
y")
("C:\Users\annam\AppData\Local\Programs\MiKTeX 2.9\tex/latex/base\textcomp.sty"
("C:\Users\annam\AppData\Local\Programs\MiKTeX 2.9\tex/latex/base\ts1enc.def"
("C:\Users\annam\AppData\Local\Programs\MiKTeX 2.9\tex/latex/base\ts1enc.dfu"))
) ("C:\Users\annam\AppData\Local\Programs\MiKTeX 2.9\tex/latex/base\inputenc.st

<Figure size 388.543x240.133 with 4 Axes>